# 데이터 추가 및 데이터셋 정리

In [4]:
import os, sys
import pandas as pd
import numpy as np
from numpy import loadtxt
from collections import deque

## 1. 추가된 앉는 모션 데이터들을 하나의 txt파일로 만들기
* 더불어, 해당 데이터의 y 라벨링 추가함
* [참고] 라벨링 번호 의미

1.   걷기 (위로 계단오르기, 아래로 계단 오르기 모두)
2.   서있기
3.   눕기
4.   앉기 (섰다가 앉기, 앉았다가 서기 모두)

In [18]:
# 각각의 앉기(label = 4) 파일을 하나의 txt 파일로 묶기 
file_path = '/content/drive/MyDrive/motion_dataset/'
sitting_path = file_path + 'sitting/'

# sitting 폴더 내 모든 txt파일 이름 sitlist에 저장
sitlist = os.listdir(sitting_path)
sitlist = sitlist [:-1] # 마지막에 .ipynb 파일 하나가 생성되었길래 제거.

# blabla_42.txt가 마지막임을 확인
sitlist[-1]

'trainx_sit_all.txt'

In [ ]:
# 모든 txt를 불러오면서 하나의 array로 concatenate 하기
datalist = np.empty((1,6))  # 빈 다차원 배열 만들기 ( = [] 하면 에러남)
for i in sitlist:
  data = np.loadtxt(sitting_path + i)
  datalist = np.concatenate((datalist,data))

# 맨 처음 empty 행은 지워주기
# 최종 shape이 (14858, 6)
# 첫 행의 모양이 [ 1.92,-9.18999,-0.13999,0.51969296,1.1646136,0.24540055]
# 이면 성공
datalist = np.delete(datalist, (0), axis = 0)
print(datalist.shape)
print(datalist)

In [ ]:
# 앉기 데이터 x 저장
np.savetxt(sitting_path + 'trainx_sit_all.txt', datalist, delimiter=" ")

# # 앉기 데이터 제대로 저장되었는지 다시 열어서 확인
# # google drive에도 제대로 저장되었음 확인(2022-01-24)
# temp = np.loadtxt(sitting_path + 'trainx_sit_all.txt', delimiter=" ")
# print(temp)
# print(temp.shape)

In [23]:
# 모든 행마다 마지막 열에 앉기 데이터임을 표시하는 label = 4 넣기
datalist = np.loadtxt(sitting_path + 'trainx_sit_all.txt', delimiter=" ")
temp = np.full((14858,1), 4, dtype='int')
datalist = np.append(datalist, temp, axis = 1)
print(datalist.shape)
print(datalist)

(14858, 7)
[[ 1.92       -9.18999    -0.13999    ...  1.1646136   0.24540055
   4.        ]
 [ 1.93       -9.26999    -0.15999    ...  1.1564646   0.15576075
   4.        ]
 [ 1.93       -9.34999    -0.18999    ...  1.1768372   0.05389733
   4.        ]
 ...
 [ 7.66       -5.85999     4.51       ... -3.8226192   5.9334536
   4.        ]
 [ 7.8        -5.88999     4.48       ... -3.6107433   6.389802
   4.        ]
 [ 7.98       -5.92999     4.45       ... -2.7387924   6.952088
   4.        ]]


In [11]:
# 저장하기
np.savetxt(sitting_path+'train_sit_xy.txt',datalist, delimiter=" ")

## 2. 기존 x 데이터 옆으로 y 열 넣기
* 기존 데이터 2차원으로 변경 + 라벨 열column 추가

In [12]:
# 기존 raw 데이터 열기
train_path = file_path + 'train_dataset/'
accx = np.loadtxt(train_path+'acc_X.txt')
accy = np.loadtxt(train_path+'acc_Y.txt')
accz = np.loadtxt(train_path+'acc_Z.txt')
gyrox = np.loadtxt(train_path+'gyro_X.txt')
gyroy = np.loadtxt(train_path+'gyro_Y.txt')
gyroz = np.loadtxt(train_path+'gyro_Z.txt')
answers = np.loadtxt(train_path+'Answer.txt')

In [13]:
# 기존 raw 데이터들 -> 하나의 다중배열로 병합
my_data = [accx, accy, accz, gyrox, gyroy, gyroz]
my_data = map(np.ravel, my_data)
dataset = np.column_stack(my_data)
print(dataset.shape) #(3703200, 6)
print(answers.shape) #(18516,)<- dataset이 (18516, 200, 6) 3차원 배열일 때로 맞춰서 만들어진 answers

(3703200, 6)
(18516,)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  This is separate from the ipykernel package so we can avoid doing imports until


In [27]:
# answers 길이 3703200으로 변경
new_answers=np.array([elem for elem in answers for j in range(200)], dtype='int')
new_answers=new_answers.reshape((-1,1)) # More simpler way with python syntax sugar 
print(new_answers.shape)
print(new_answers)

# 작은 예제로 확인
# y=[1,2,3,4]
# b=np.array([elem for elem in y  for j in range(5)])
# b # array([1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4])

# 또 다른 방법
# b = np.array([[i]*200 for i in y]).flatten()

(3703200, 1)
[[1]
 [1]
 [1]
 ...
 [1]
 [1]
 [1]]


In [28]:
#기존 데이터 묶음 dataset의 새로운 열로 new_asnwers 붙이기
dataset = np.append(dataset, new_answers, axis = 1)
dataset

array([[  2.35      , -11.24999   ,  -0.05999   , ...,  -3.8429918 ,
          2.6371534 ,   1.        ],
       [  2.56      , -11.60999   ,  -0.10999   , ...,  -3.1503205 ,
          2.2663705 ,   1.        ],
       [  2.64      , -11.71999   ,  -0.11999   , ...,  -2.865103  ,
          2.2134016 ,   1.        ],
       ...,
       [ 11.22      ,   3.03      ,  -3.53999   , ...,  -2.3394878 ,
          0.29022044,   1.        ],
       [ 11.65      ,   3.13      ,  -3.90999   , ...,  -2.1683574 ,
          0.35948756,   1.        ],
       [ 12.04      ,   3.18      ,  -4.23999   , ...,  -1.9279597 ,
          0.35948756,   1.        ]])

## 3. 기존 데이터셋과 앉기 데이터셋 결합
* 위의 'dataset'과, 1.에서 작업한 앉기 데이터 'datalist'('train_sit_xy.txt')를 행 방향으로 병합

In [29]:
finalset = np.concatenate((dataset, datalist), axis = 0)
print(finalset.shape) #기존데이터(3703200,7) + 새 데이터(14858,7) = (3718058,7)
print(finalset)

(3718058, 7)
[[  2.35      -11.24999    -0.05999   ...  -3.8429918   2.6371534
    1.       ]
 [  2.56      -11.60999    -0.10999   ...  -3.1503205   2.2663705
    1.       ]
 [  2.64      -11.71999    -0.11999   ...  -2.865103    2.2134016
    1.       ]
 ...
 [  7.66       -5.85999     4.51      ...  -3.8226192   5.9334536
    4.       ]
 [  7.8        -5.88999     4.48      ...  -3.6107433   6.389802
    4.       ]
 [  7.98       -5.92999     4.45      ...  -2.7387924   6.952088
    4.       ]]


In [33]:
#저장
np.savetxt(train_path+'dataset_withlabels.txt', finalset, delimiter=" ")
os.listdir(train_path)

['Answer.txt',
 'acc_X.txt',
 'gyro_X.txt',
 'gyro_Z.txt',
 'gyro_Y.txt',
 'acc_Y.txt',
 'acc_Z.txt',
 'dataset_withlabels.txt']